In [ ]:

!pip install transformers[torch] gradio pandas matplotlib --quiet

import os
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import gradio as gr
from google.colab import drive
from tqdm.notebook import tqdm
ASPECTS = ["acting", "dialogues", "storyline", "characters", "cinematography", "visual effects", "direction"]
ASPECT_CONFIG = {
    "acting": {"threshold": 0.65, "keywords": ["act", "perform"]},
    "dialogues": {"threshold": 0.6, "keywords": ["dialogue", "script"]},
    "storyline": {"threshold": 0.55, "keywords": ["story", "plot"]},
    "characters": {"threshold": 0.5, "keywords": ["character", "role"]},
    "cinematography": {"threshold": 0.45, "keywords": ["camera", "shot"]},
    "visual effects": {"threshold": 0.4, "keywords": ["effect", "vfx"]},
    "direction": {"threshold": 0.6, "keywords": ["director", "vision"]}
}
BATCH_SIZE = 16
MAX_LENGTH = 128
EPOCHS = 2
LEARNING_RATE = 4e-5

def load_sample_data():
    """Load a smaller subset of data for faster processing"""
    data = pd.read_csv("IMDB Dataset.csv", nrows=2000)
    data['sentiment'] = data['sentiment'].map({'positive': 1, 'negative': 0})

    for aspect in ASPECTS:
        pattern = '|'.join(ASPECT_CONFIG[aspect]["keywords"])
        data[aspect] = data['review'].str.contains(pattern, case=False).astype(int)

    return data

data = load_sample_data()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

class FastAspectDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float)
        }

dataset = FastAspectDataset(
    data['review'].values,
    data[ASPECTS].values,
    tokenizer,
    MAX_LENGTH
)

train_size = int(0.8 * len(dataset))
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(ASPECTS),
    problem_type="multi_label_classification"
).to(device)
scaler = torch.cuda.amp.GradScaler()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

model.train()
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    for batch in tqdm(train_loader, desc="Training"):
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(
                input_ids=batch['input_ids'].to(device),
                attention_mask=batch['attention_mask'].to(device),
                labels=batch['labels'].to(device)
            )
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    print(f"Epoch {epoch + 1} Loss: {loss.item():.4f}")

@torch.no_grad()
def predict_aspect_sentient(review):
    model.eval()
    inputs = tokenizer(
        review,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    ).to(device)

    outputs = model(**inputs)
    probs = torch.sigmoid(outputs.logits).cpu().numpy()[0]

    results = {}
    for i, aspect in enumerate(ASPECTS):
        prob = probs[i]
        threshold = ASPECT_CONFIG[aspect]["threshold"]
        results[aspect] = {
            'sentiment': 'Positive' if prob > threshold else 'Negative',
            'confidence': float(prob if prob > threshold else 1 - prob),
            'probability': float(prob)
        }

    return results

def create_fast_plot(results):
    fig, ax = plt.subplots(figsize=(10, 5))
    aspects = list(results.keys())
    confidences = [res['confidence'] for res in results.values()]
    colors = ['green' if res['sentiment'] == 'Positive' else 'red' for res in results.values()]

    bars = ax.bar(aspects, confidences, color=colors)
    ax.set_title('Aspect Sentiment Analysis', pad=10)
    ax.set_ylim(0, 1)

    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}',
                ha='center', va='bottom')

    plt.tight_layout()
    return fig

def analyze_review(review):
    if not review.strip():
        return "Please enter a review", None

    results = predict_aspect_sentient(review)

    text_output = "Results:\n" + "\n".join(
        f"{k}: {v['sentiment']} ({v['confidence']:.2f})"
        for k, v in results.items()
    )

    fig = create_fast_plot(results)
    fig.savefig('temp_plot.png', bbox_inches='tight')
    plt.close(fig)

    return text_output, 'temp_plot.png'

iface = gr.Interface(
    fn=analyze_review,
    inputs=gr.Textbox(label="Review", lines=5),
    outputs=[
        gr.Textbox(label="Analysis"),
        gr.Image(label="Visualization")
    ],
    title="Fast Aspect Sentiment Analysis",
    examples=[
        ["Great acting and story but poor visual effects"],
        ["Excellent direction makes up for weak dialogues"]
    ]
)

iface.launch(debug=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.7 MB/s eta 0:00:00


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2


<ipython-input-2-c207dcc236fb>:103: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training:   0%|          | 0/100 [00:00<?, ?it/s]

<ipython-input-2-c207dcc236fb>:113: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 1 Loss: 0.5213
Epoch 2/2


Training:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch 2 Loss: 0.4194
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a0ee68ae8d1338a668.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
